In [28]:
import os
from urllib import request
from src.ML_ops_end_end_ import logger
import zipfile


In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str 
    local_data_file: Path
    unzip_dir: Path

In [10]:
import os 
os.chdir("../")

In [20]:

os.chdir("MLOPS")

%pwd


'c:\\Users\\gerar\\OneDrive\\Desktop\\projects\\MLops\\MLOPS'

In [19]:
%pwd

'c:\\Users\\gerar\\OneDrive\\Desktop\\projects\\MLops'

In [30]:
import sys
from pathlib import Path

# Fix: Point to the src/ inside MLOPS/
src_path = Path().resolve() / "MLOPS" / "src"
sys.path.append(str(src_path))

# Confirm
print("src exists:", src_path.exists())
print("module exists:", (src_path / "ML_ops_end_end_" / "constants" / "__init__.py").exists())


src exists: False
module exists: False


In [32]:
from ML_ops_end_end_.constants import *
from ML_ops_end_end_.utils.common import read_yaml ,  create_directories


In [33]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config



In [34]:
##component - data ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    #downlaoding the file from the source URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with headers:\n  {headers}")

        else:
            logger.info("file already exists")


    #unzip the file
    def extract_zip_file(self):
      """
          zip_file_path: str
          extracts the zip file into the data directory 
          function returns none
      """
      unzip_path = self.config.unzip_dir
      os.makedirs(unzip_path, exist_ok=True)
      with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
          zip_ref.extractall(unzip_path)
      




In [36]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    logger.exception(e)
    raise e


[2025-07-12 09:58:27,162 - INFO - common  - yaml file: config\config.yaml loaded successfully]
[2025-07-12 09:58:27,164 - INFO - common  - yaml file: params.yaml loaded successfully]
[2025-07-12 09:58:27,165 - INFO - common  - yaml file: schema.yaml loaded successfully]
[2025-07-12 09:58:27,166 - INFO - common  - created directory at: artifacts]
[2025-07-12 09:58:27,167 - INFO - common  - created directory at: artifacts/data_ingestion]
[2025-07-12 09:58:28,146 - INFO - 3148028957  - winequality-data.zip downloaded with headers:
  Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9158:303B46:3D0CF:11E439:6871E46B
Accept-Ranges: bytes
Date: Sa